# This is the lab #3.<br />I have compiled all 3 parts in one file, so please scroll down for parts 2 and 3.

## Let's start with part 1 (of 3)

### Importing libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests
import io
import lxml

### Parsing wikipedia url. Getting a substring representing the table. Doing some html tags replacements.

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
s=str(requests.get(url).content)
s1_to_find = "<table class=\"wikitable sortable\">"
s2_to_find = "</table>"

subs = s[s.find(s1_to_find) + len(s1_to_find):s.find(s2_to_find)]
subs = subs.replace("\\n","").replace("<tbody>","").replace("</tr></tbody>","").replace("<tr>","")

sarray = subs.split("</tr>")


### Let's parse each html row to get its items: <b>Postal Cod</b>, <b>Borough</b>, and <b>Neighborhood</b>
#### We will drop <u><i>Not assigned</i></u> 

In [3]:
indexPost = 1
indexBor = 2
indexNeigh = 3

df = pd.DataFrame({"Postal Code" : [], "Borough" : [], "Neighborhood" : []} )
for i in range(1, len(sarray)):
    retList = sarray[i].replace("</td>","").split("<td>")
    if retList[indexBor] != "Not assigned":
        df = df.append({"Postal Code" : retList[indexPost],"Borough" : retList[indexBor], "Neighborhood" : retList[indexNeigh]}, ignore_index = True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen\'s Park, Ontario Provincial Government"


### Finally, let's see the dataframe shape

In [4]:
df.shape

(103, 3)

## Ok, part 1 completed. Let's do the part 2 (of 3)

### Importing libraries

In [5]:
import geocoder # import geocoder

In [6]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
#for indCodes in df["Postal Code"]:
#    while(lat_lng_coords is None):
#        g = geocoder.google("Toronto, Ontario, M5G")
#        lat_lng_coords = g.latlng
#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]


#### Unfortunately, geocoder doesn't work. So let's proceed with csv. Loading data first.

In [7]:
url_csv = 'https://cocl.us/Geospatial_data'
df_latlong = pd.read_csv(url_csv)
df_latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Now let's merge df's

In [8]:
dfmerge = df.merge(df_latlong, on="Postal Code")
dfmerge.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen\'s Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
dfmerge.shape

(103, 5)

## Ok, part 2 completed. Let's do the part 3 (of 3)

### Importing libs

In [10]:
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### KMeans for 'kclusters' clusters on the data

In [11]:
# set number of clusters
kclusters = 7
dfKM = dfmerge.drop('Neighborhood', 1).drop('Borough', 1).drop('Postal Code', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfKM)

# check cluster labels generated for each row in the dataframe
kmeans.cluster_centers_

array([[ 43.66680592, -79.37406473],
       [ 43.63776628, -79.53356934],
       [ 43.78769261, -79.22129714],
       [ 43.76524349, -79.39441482],
       [ 43.72998549, -79.29644876],
       [ 43.68566194, -79.44295946],
       [ 43.72751776, -79.53223569]])

### Let's make a map!

In [12]:
# create map
map_clusters = folium.Map(location=[43.6499487,-79.3878643], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], range(kclusters)):
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup='Cluster ' + str(cluster+1),
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Ok, part 3 completed!